## Testing Cassandra
En este notebook se realizarán las pruebas de conexión y inserción de datos en Cassandra así como las consultas de datos.
<div align="center">
    <img src="img/initial_table.png" width="500">
</div>

In [22]:
#imports
from cassandra.cluster import Cluster
import time
import pandas as pd

Conexión a Cassandra

In [61]:
def wait_for_cassandra():
    while True:
        try:
            cluster = Cluster(['cassandra'])
            session = cluster.connect()
            return cluster, session
        except Exception as e:
            print(f"Espere por favor... Se está cargando Cassandra: {e}")
            time.sleep(20)

In [75]:
def measure_request_time(func, *args, **kwargs):
    """Mide el tiempo de ejecución de una función."""
    start_time = time.time()  # Iniciar el temporizador
    result = func(*args, **kwargs)  # Ejecutar la función
    end_time = time.time()  # Finalizar el temporizador
    duration = end_time - start_time  # Calcular la duración
    print(f"Tiempo de ejecución: {duration:.4f} segundos")
    return result

In [80]:
def fetch_all_users(session):
    """Recupera todos los usuarios de la base de datos y los muestra en una tabla estilo Excel."""
    try:
        # Ejecutar la consulta para obtener todos los usuarios
        rows = session.execute("SELECT * FROM user_data.users")
        
        # Convertir los resultados a un DataFrame
        users_df = pd.DataFrame(rows)
        
        # Mostrar los datos en formato estilo Excel
        print("\nDatos de usuarios:")
        styled_df = users_df.style.set_table_attributes('style="width:100%; border-collapse:collapse;"') \
                                   .set_table_styles([{'selector': 'th', 'props': [('background-color', '#f2f2f2'), ('font-weight', 'bold')]}]) \
                                   .set_properties(**{'border': '1px solid black'}) \
        
        # Mostrar el DataFrame estilizado
        display(styled_df)
        
    except Exception as e:
        print(f"Error al recuperar datos: {e}")

# Intentar ejecutar la acción de recuperación de usuarios
measure_request_time(fetch_all_users, session)



Datos de usuarios:


,id,email,first_name,gender,last_name
0,5,dseeler4@usda.gov,Debbi,Female,Seeler
1,10,npaulich9@wikispaces.com,Nadean,Female,Paulich
2,16,bmeadef@elegantthemes.com,Brandie,Genderqueer,Meade
3,13,bambrusc@boston.com,Bertha,Female,Ambrus
4,11,oschneidara@storify.com,Olympia,Female,Schneidar
5,1,vegetta_email@example.com,Athena,Female,Elijahu
6,19,tsteeli@blinklist.com,Tybi,Female,Steel
7,8,mmion7@issuu.com,Maddie,Male,Mion
8,2,spaladini1@who.int,Selinda,Female,Paladini
9,4,pharvett3@uiuc.edu,Perry,Male,Harvett


Tiempo de ejecución: 0.0140 segundos


In [85]:
def get_user_by_id(session, user_id):
    """Recupera un usuario por su ID desde la base de datos."""
    try:
        result = session.execute("""
            SELECT * FROM user_data.users WHERE id = %s
        """, (user_id,))
        
        if result:
            for row in result:
                print(f"ID: {row.id}, Nombre: {row.first_name} {row.last_name}, Email: {row.email}, Género: {row.gender}")
        else:
            print("Usuario no encontrado.")
    
    except Exception as e:
        print(f"Error al recuperar el usuario: {e}")


In [87]:
def update_user_email(session, user_id, new_email):
    try:
        session.execute("""
            UPDATE user_data.users
            SET email = %s
            WHERE id = %s
        """, (new_email, user_id))
        print(f"Email actualizado para el usuario con ID {user_id}.")
    except Exception as e:
        print(f"Error al actualizar el email: {e}")


cluster, session = wait_for_cassandra()

measure_request_time(update_user_email, session, 1, "vegetta_email@example.com")
measure_request_time(get_user_by_id, session, 1)

Email actualizado para el usuario con ID 1.
Tiempo de ejecución: 0.0020 segundos
ID: 1, Nombre: Athena Elijahu, Email: vegetta_email@example.com, Género: Female
Tiempo de ejecución: 0.0027 segundos


In [90]:
def get_emails(session):
    """Recupera los correos electrónicos de los usuarios desde la base de datos."""
    try:
        rows = session.execute("SELECT email FROM user_data.users")
        emails = [row.email for row in rows]
        print(emails)
    
    except Exception as e:
        print(f"Error al ejecutar la función: {e}")
        return []

measure_request_time(get_emails, session)


['dseeler4@usda.gov', 'npaulich9@wikispaces.com', 'bmeadef@elegantthemes.com', 'bambrusc@boston.com', 'oschneidara@storify.com', 'vegetta_email@example.com', 'tsteeli@blinklist.com', 'mmion7@issuu.com', 'spaladini1@who.int', 'pharvett3@uiuc.edu', 'akobierieckih@tuttocitta.it', 'tablee@disqus.com', 'draddonj@blogs.com', 'tquilkin6@usatoday.com', 'rwilcockes5@chicagotribune.com', 'agobel8@yahoo.co.jp', 'dporsond@seattletimes.com', 'twhillockg@oracle.com', 'mkeavenyb@omniture.com', 'csousa2@imgur.com']
Tiempo de ejecución: 0.0048 segundos
